CHECKS TO MAKE/THINGS TO LOOK INTO:
- Look into leagues that with missing data from the Matchup Data and leagues with missing data from the Projected Points data
    a) None of the leagues in the Matchup data have week 17 data and only a few hundred have week 16 data. This          shouldn't be an issue since we only need data from the first week of playoffs (which should be week 15 at          the latest), but it is a little odd.
        1. Only ~60 leagues in the Matchup data have missing data for weeks 1-15
    b) 
- Check that the scores from the Matchup and Projected data match

In [43]:
import pandas as pd
import numpy as np
import os

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

LEAGUE_DATA_DIRECTORY = "/home/cdelong/Python-Projects/FF-Web-App/Simulation-Data/"


In [79]:
#####################################################################################
################################# General Functions #################################
#####################################################################################

def get_league_ids(leagues_found_csv='Leagues_Found.csv'):
    """ Returns dataframe containing all of the league IDs found """

    df_leagues_found = pd.read_csv(leagues_found_csv)

    df_leagues_found = df_leagues_found.loc[df_leagues_found['seasonId'] == 2020]

    league_ids = list(df_leagues_found['leagueId'])
    
    return league_ids


def convert_tuple_to_list(tuple_var):
    """ 
    Converts tuple to a list
    Note: This isn't really necessary, but accounts for 1D tuple cases so i'm using it
    """
    
    list_var = []
    
    # Need to process 1D and 2D tuples differently
    if type(tuple_var[0]) is tuple:
        for value in tuple_var:

            dict_key = value[0]
            dict_value = value[1]

            list_var.append([dict_key, dict_value])
    else:
        dict_key = tuple_var[0]
        dict_value = tuple_var[1]

        list_var.append([dict_key, dict_value])
            
    return list_var


def convert_dict_to_list(dict_var):
    """ Converts dictionary to a 2D list """
    
    list_var = []
    for param, param_value in dict_var.items():
        list_var.append([param, param_value])
        
    return list_var


def rearrange_list_by_val(input_list, move_vals_list):
    """ 
    Moves the 'vals' in the 'list_var' to the front
    
    Note: This is primarily used to rearrange the columns in a dataframe. Due to this, the lists this
    will be used for will be relatively small, so I'm not worrying about efficiency
    """
    
    move_vals_list_len = len(move_vals_list)
  
    for i in range(1, move_vals_list_len + 1):
        if move_vals_list_len > 0:
            # Start with the back of the list in order to properly arrange the values
            list_index = -1 * i
        else:
            list_index = 0
            
        move_val = move_vals_list[list_index]
        
        for j, match_val in enumerate(input_list):
            if match_val == move_val:
                move_val = input_list.pop(j)
                input_list.insert(0, move_val)
            else:
                pass
                
    return input_list


def rearrange_df_columns(df, move_columns_list):
    """ Returns a dataframe with its columns rearranged according to the passed 'move_columns_list' """
    
    df = df.copy()
    
    df_cols = list(df.columns)
    
    df_cols_new = rearrange_list_by_val(df_cols, move_columns_list)
    
    df = df[df_cols_new]
    
    return df

In [46]:
def get_list_of_files(load_data_dir, extensions_list=None, include_or_exclude_ext=None):
    """ Returns a list containing all of the excel files found in the load_data_dir """
    # Pulling the original working directory in order to reset it later
    
    if extensions_list is None:
        extensions_list = []
 
    if include_or_exclude_ext is None:
        include_or_exclude_ext = "include"
        
    # This factor determines if the file being checked needs to be included or excluded from the list
    if include_or_exclude_ext == "include":
        include_exclude_factor = 1
    else:
        include_exclude_factor = -1
        
    original_working_directory = os.getcwd()
    
    os.chdir(load_data_dir)
    
    files_list = os.listdir()
    
    os.chdir(original_working_directory)
    
    remove_files = []
    for file in files_list:
        remove_file_bool = 1 * include_exclude_factor
        
        # Checks if the file has any of the extensions in the list
        for ext in extensions_list: 
            file_index = len(ext) * -1
            if file[file_index:] == ext:
                # Override the flag if the file is found
                remove_file_bool = -1 * include_exclude_factor
                break
            
        if remove_file_bool == 1:
            remove_files.append(file)
            
    for remove_file in remove_files:
        files_list.remove(remove_file)
            
    return files_list


def read_in_data(load_data_dir):
    """ Returns a dictionary containing every dataframe created from each csv found in the load_data_dir """
    
    file_list = get_list_of_files(load_data_dir, extensions_list=['.csv'])
    
    raw_df_dict = {}
    for file in file_list:
        df_name_index = file.find('_')
        
        if df_name_index > -1:
            df_name = file[:df_name_index]
        else: 
            df_name = file

        full_file_path = LEAGUE_DATA_DIRECTORY + file

        raw_df_dict[df_name] = pd.read_csv(full_file_path)
        
    return raw_df_dict
        
# raw_df_dict = read_in_data(LEAGUE_DATA_DIRECTORY)


In [130]:

def merge_proj_and_matchup_data(proj_data, matchup_data):
    """ Returns a dataframe that mergeds the proj_data and matchup_data"""

    # Aggregate the proj_data to be on the same level as the matchup_data
    by_group = ['league_id', 'Week', 'Team']
    agg_proj_points_data = proj_data.groupby(by_group, as_index=False)['Actual'].sum()

    keep_vars = ['season_id', 'league_id', 'week_number', 'teamId', 'score']
    matchup_data = matchup_data[keep_vars]

    left_on_list=['league_id', 'Week', 'Team']
    right_on_list=['league_id', 'week_number', 'teamId']
    df = pd.merge(agg_proj_points_data, matchup_data,
                                        left_on=left_on_list, right_on=right_on_list, how='outer')

    rename_dict = {'Actual': 'ProjPointsData_Score', 'score': 'MatchupData_Score'}
    df.rename(columns=rename_dict, inplace=True)
    
    # Update the week number and team ids for records with missing matchup data
    df['week_number'].loc[df['week_number'].isnull()] = df['Week']
    df['teamId'].loc[df['teamId'].isnull()] = df['Team']

    drop_vars = ['Week', 'Team', 'season_id']
    df.drop(columns=drop_vars, inplace=True)
    
    return df


def add_vars(df):
    """ Adds variables to the merged ProjectPointsData and MatcupData """
    
    df = df.copy()
    
    # Add flags that indicate there is data for each df that was merged
    df['ProjPointsData_Missing'] = 0
    df['ProjPointsData_Missing'].loc[
        df['ProjPointsData_Score'].isnull()] = 1
    
    df['MatchupData_Missing'] = 0
    df['MatchupData_Missing'].loc[
    df['MatchupData_Score'].isnull()] = 1
    
    return df


def check_missing_data(df, missing_data_flag):
    
    df = df[df[missing_data_flag] == 1]
    
    return df
    

proj_points_data_starters = raw_df_dict['ProjPointsData'][proj_points_data['Pos'] != 'Bench']
compare_proj_matchup_data = merge_proj_and_matchup_data(proj_points_data_starters, 
                                                        raw_df_dict['MatchupData'])

compare_proj_matchup_data = add_vars(compare_proj_matchup_data)

rearrange_cols = ['season_id', 'league_id', 'week_number', 
                  'teamId', 'ProjPointsData_Score', 'MatchupData_Score']
compare_proj_matchup_data = rearrange_df_columns(compare_proj_matchup_data, rearrange_cols)

compare_proj_matchup_data

,league_id,week_number,teamId,ProjPointsData_Score,MatchupData_Score,ProjPointsData_Missing,MatchupData_Missing
0,24693394,1.0,1.0,151.50,151.50,0,0
1,24693394,1.0,2.0,151.50,151.50,0,0
2,24693394,1.0,3.0,105.84,105.84,0,0
3,24693394,1.0,4.0,104.40,104.40,0,0
4,24693394,1.0,5.0,124.38,124.38,0,0
...,...,...,...,...,...,...,...
358300,61643607,13.0,6.0,NaN,223.10,1,0
358301,61643607,13.0,7.0,NaN,263.50,1,0
358302,61643607,13.0,8.0,NaN,216.20,1,0
358303,61643607,13.0,9.0,NaN,173.74,1,0


In [156]:
temp_dir = LEAGUE_DATA_DIRECTORY + "Check/"
file_name = 'check.csv'
full_path = temp_dir + file_name

pd.set_option('display.max_rows', 2000)

missing_flag = 'ProjPointsData_Missing'
# missing_flag = 'MatchupData_Missing'
input_data = compare_proj_matchup_data.loc[compare_proj_matchup_data['week_number'] < 16]


missing_proj_data = check_missing_data(input_data, missing_flag)
missing_proj_data = missing_proj_data.groupby(['league_id', 'week_number'], as_index=False).size()

missing_proj_data = missing_proj_data.pivot(index='league_id', columns='week_number', values='size')
print(len(missing_proj_data))

missing_proj_data

108


week_number,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
league_id,,,,,,,,,,,,,,,
24695946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
24712271,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
24753044,12.0,12.0,11.0,11.0,12.0,11.0,12.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,NaN
24757703,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
24765603,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
24812924,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,NaN,NaN
24828696,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
24831845,11.0,11.0,11.0,12.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,12.0,11.0
24836621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [153]:

# check_proj_data = raw_df_dict['ProjPointsData']
# check_proj_data[check_proj_data['league_id'] == 24695946]

check_merge_data = compare_proj_matchup_data
check_merge_data[check_merge_data['league_id'] == 24808253]


# check_missing_proj = compare_proj_matchup_data[compare_proj_matchup_data['league_id'] == 24765603]
# check_missing_proj.to_csv(full_path, index=False)


,league_id,week_number,teamId,ProjPointsData_Score,MatchupData_Score,ProjPointsData_Missing,MatchupData_Missing
6129,24808253,1.0,1.0,98.78,0.0,0,0
6130,24808253,1.0,2.0,116.10,0.0,0,0
6131,24808253,1.0,3.0,143.60,0.0,0,0
6132,24808253,1.0,4.0,97.10,0.0,0,0
6133,24808253,1.0,5.0,114.84,0.0,0,0
6134,24808253,1.0,6.0,109.06,0.0,0,0
6135,24808253,1.0,7.0,101.68,0.0,0,0
6136,24808253,1.0,8.0,81.32,0.0,0,0
6137,24808253,1.0,9.0,108.00,0.0,0,0
6138,24808253,1.0,10.0,111.00,0.0,0,0


In [71]:
raw_df_dict['MatchupData']

,season_id,league_id,week_number,macthupPeriodId,teamId,teamId_opp,score,score_opp,home_or_away,regular_season_ind,playoffTierType
0,2020,24693394,1,1,1,5,151.50,124.38,away,1,NONE
1,2020,24693394,1,1,2,6,151.50,141.88,home,1,NONE
2,2020,24693394,1,1,3,4,105.84,104.40,home,1,NONE
3,2020,24693394,1,1,4,3,104.40,105.84,away,1,NONE
4,2020,24693394,1,1,5,1,124.38,151.50,home,1,NONE
...,...,...,...,...,...,...,...,...,...,...,...
320916,2020,68999315,15,14,7,11,144.04,217.28,home,0,LOSERS_CONSOLATION_LADDER
320917,2020,68999315,15,14,8,5,150.76,255.84,home,0,WINNERS_CONSOLATION_LADDER
320918,2020,68999315,15,14,9,-1,242.98,0.00,home,0,LOSERS_CONSOLATION_LADDER
320919,2020,68999315,15,14,10,3,194.12,165.42,home,0,LOSERS_CONSOLATION_LADDER
